In [1]:
import sys
sys.path.insert(1, '../')
import torch
import models.resnet

In [2]:
state = torch.load('../pretrained_models/resnet110-1d1ed7c2.th')

In [3]:
replace_list = [
    ('module.conv1','pre_layer.0'),
    ('module.bn1',  'pre_layer.1'),
    ('module.linear',  'classifier.2'),
    ('module.', ''),
    ('conv1', 'conv_norm_act_1.0'),
    ('conv2', 'conv_norm_act_2.0'),
    ('bn1', 'conv_norm_act_1.1'),
    ('bn2', 'conv_norm_act_2.1'),
    ('block1','0'),
    ('block2','1'),
    ('block3','2'),
]

def to_new_key(key):
    for o,n in replace_list:
        key = key.replace(o,n)
    return key

In [4]:
model = models.resnet.resnet110()

In [5]:
from collections import OrderedDict
converted_state = OrderedDict({to_new_key(k):v for k,v in state['state_dict'].items()})

In [6]:
model.load_state_dict(converted_state) 

<All keys matched successfully>

In [8]:
import numpy as np
from helpers.loss_accuracy import accuracy
from helpers.data import get_dataloaders
model = model.cuda().eval()
np.mean([accuracy(model(x.cuda()).cpu(), y) for x,y in get_dataloaders('cifar10', 32)[1]])

Files already downloaded and verified


0.93680114